<a href="https://colab.research.google.com/github/lamia308/Demo/blob/master/Licence_Plates_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np

# Load YOLOv4 configuration and weights
net = cv2.dnn.readNet("yolov4.cfg", "yolov4.weights")

# Load class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set input image size
input_size = (608, 608)

# Load image
image = cv2.imread("image.jpg")

# Resize image to input size
resized_image = cv2.resize(image, input_size)

# Normalize image and convert it to blob
blob = cv2.dnn.blobFromImage(resized_image, 1 / 255.0, input_size, swapRB=True, crop=False)

# Set the input blob for the network
net.setInput(blob)

# Perform forward pass
outputs = net.forward()

# Filter out the bounding boxes with confidence score above threshold
conf_threshold = 0.5
nms_threshold = 0.4

boxes = []
confidences = []
class_ids = []

for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > conf_threshold and class_id == 2:
            center_x = int(detection[0] * input_size[0])
            center_y = int(detection[1] * input_size[1])
            width = int(detection[2] * input_size[0])
            height = int(detection[3] * input_size[1])

            x = int(center_x - width / 2)
            y = int(center_y - height / 2)

            boxes.append([x, y, width, height])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maxima suppression to remove overlapping bounding boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# Iterate over the remaining indices
for i in indices:
    i = i[0]
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]

    # Draw bounding box and display class label
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, classes[class_ids[i]], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the result
cv2.imshow("Vehicle Number Plate Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: ignored

In [2]:
import cv2
import numpy as np

# Load YOLOv4 configuration and weights
net = cv2.dnn.readNet("yolov4.cfg", "yolov4.weights")

# Load class names
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set input image size
input_size = (608, 608)

# Function to perform object detection on image
def detect_objects_image(image):
    # Resize image to input size
    resized_image = cv2.resize(image, input_size)

    # Normalize image and convert it to blob
    blob = cv2.dnn.blobFromImage(resized_image, 1 / 255.0, input_size, swapRB=True, crop=False)

    # Set the input blob for the network
    net.setInput(blob)

    # Perform forward pass
    outputs = net.forward()

    # Filter out the bounding boxes with confidence score above threshold
    conf_threshold = 0.5
    nms_threshold = 0.4

    boxes = []
    confidences = []
    class_ids = []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > conf_threshold and class_id == 2:
                center_x = int(detection[0] * input_size[0])
                center_y = int(detection[1] * input_size[1])
                width = int(detection[2] * input_size[0])
                height = int(detection[3] * input_size[1])

                x = int(center_x - width / 2)
                y = int(center_y - height / 2)

                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Iterate over the remaining indices
    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box[0], box[1], box[2], box[3]

        # Draw bounding box and display class label
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, classes[class_ids[i]], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image

# Function to perform object detection on video
def detect_objects_video(video_path):
    # Open video file
    video = cv2.VideoCapture(video_path)

    # Get video dimensions
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    # Create output video writer
    output_video = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    while True:
        ret, frame = video.read()

        if not ret:
            break

        # Perform object detection on frame
        detected_frame = detect_objects_image(frame)

        # Write the detected frame to the output video
        output_video.write(detected_frame)

        # Display the resulting frame
        cv2.imshow("Vehicle Number Plate Detection", detected_frame)

        if cv2.waitKey(1) == ord("q"):
            break

    # Release video capture and writer
    video.release()
    output_video.release()
    cv2.destroyAllWindows()

# Process image
image = cv2.imread("image.jpg")
detected_image = detect_objects_image(image)
cv2.imshow("Vehicle Number Plate Detection", detected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Process video
video_path = "video.mp4"
detect_objects_video()

SyntaxError: ignored